# Ladder Network with Convolutional Layers
**Input**: M,N,K specifying the general architecture: INPUT -> [[CONV -> RELU]*N -> POOL]*M -> [FC -> RELU]*K -> FC

The Ladder network typically has a following layer structure: [Input size, say 700, 1000, 500, 250, 250, 250, 10]

# Data

In [4]:
!pip install attributedict
import numpy as np
from sklearn.decomposition import PCA
import scipy.io as sio
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import os
import random
from random import shuffle
from skimage.transform import rotate
import scipy.ndimage
from sklearn.model_selection import train_test_split
import scipy

def load_pavia():
  
  !pip install GoogleDriveDownloader
  from google_drive_downloader import GoogleDriveDownloader as gdd
  gdd.download_file_from_google_drive(file_id='146WN2eZ6Syf-z1KMVRw9GmZdBu_g1JBj',
                                    dest_path='./datasets/paviau.mat', unzip=False)

  gdd.download_file_from_google_drive(file_id='1L9OoAHnLVmPGbfKx8NhEbugxMzE1PG4j',
                                    dest_path='./datasets/paviau_gt.mat', unzip=False)

  X = sio.loadmat('./datasets/paviau.mat')['paviaU']
  y = sio.loadmat('./datasets/paviau_gt.mat')['paviaU_gt']

  return X, y
  
  
def createPatches(X, y, windowSize=5, removeZeroLabels = True):
  margin = int((windowSize - 1) / 2)
  zeroPaddedX = padWithZeros(X, margin=margin)
  # split patches
  patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]))
  patchesLabels = np.zeros((X.shape[0] * X.shape[1]))
  patchIndex = 0
  for r in range(margin, zeroPaddedX.shape[0] - margin):
      for c in range(margin, zeroPaddedX.shape[1] - margin):
          patch = zeroPaddedX[r - margin:r + margin + 1, c - margin:c + margin + 1]   
          patchesData[patchIndex, :, :, :] = patch
          patchesLabels[patchIndex] = y[r-margin, c-margin]
          patchIndex = patchIndex + 1
  if removeZeroLabels:
      patchesData = patchesData[patchesLabels>0,:,:,:]
      patchesLabels = patchesLabels[patchesLabels>0]
      patchesLabels -= 1
  return patchesData, patchesLabels
  
  
def padWithZeros(X, margin=2):
  newX = np.zeros((X.shape[0] + 2 * margin, X.shape[1] + 2* margin, X.shape[2]))
  x_offset = margin
  y_offset = margin
  newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X
  return newX
  
def standartizeData(X):
  newX = np.reshape(X, (-1, X.shape[2]))
  scaler = preprocessing.StandardScaler().fit(newX)  
  newX = scaler.transform(newX)
  newX = np.reshape(newX, (X.shape[0],X.shape[1],X.shape[2]))
  return newX, scaler
  
  
def applyPCA(X, numComponents=75):
  newX = np.reshape(X, (-1, X.shape[2]))
  pca = PCA(n_components=numComponents, whiten=True)
  newX = pca.fit_transform(newX)
  newX = np.reshape(newX, (X.shape[0],X.shape[1], numComponents))
  return newX, pca
  
  
def diff(first, second):
  second = set(second)
  return [item for item in first if item not in second]


def splitTrainTestSet(X, y, testRatio=0.10):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testRatio, random_state=345,
                                                      stratify=y)
  return X_train, X_test, y_train, y_test
  
  
def AugmentData(X_train):
  for i in range(int(X_train.shape[0]/2)):
      patch = X_train[i,:,:,:]
      num = random.randint(0,2)
      if (num == 0):
          flipped_patch = np.flipud(patch)
      if (num == 1):
          flipped_patch = np.fliplr(patch)
      if (num == 2):
          no = random.randrange(-180,180,30)
          flipped_patch = scipy.ndimage.interpolation.rotate(patch, no,axes=(1, 0),
                                                             reshape=False, output=None, order=3, mode='constant', cval=0.0, prefilter=False)

      patch2 = flipped_patch
      X_train[i,:,:,:] = patch2

  return X_train
  
  
def oversampleWeakClasses(X, y):
  uniqueLabels, labelCounts = np.unique(y, return_counts=True)
  maxCount = np.max(labelCounts)
  labelInverseRatios = maxCount / labelCounts  
  # repeat for every label and concat
  newX = X[y == uniqueLabels[0], :, :, :].repeat(round(labelInverseRatios[0]), axis=0)
  newY = y[y == uniqueLabels[0]].repeat(round(labelInverseRatios[0]), axis=0)
  for label, labelInverseRatio in zip(uniqueLabels[1:], labelInverseRatios[1:]):
      cX = X[y== label,:,:,:].repeat(round(labelInverseRatio), axis=0)
      cY = y[y == label].repeat(round(labelInverseRatio), axis=0)
      newX = np.concatenate((newX, cX))
      newY = np.concatenate((newY, cY))
  np.random.seed(seed=42)
  rand_perm = np.random.permutation(newY.shape[0])
  newX = newX[rand_perm, :, :, :]
  newY = newY[rand_perm]
  return newX, newY
  
  
  
def savePreprocessedData(X_trainPatches, X_testPatches, y_trainPatches, y_testPatches, windowSize, numPCAComponents = 0, testRatio = 0.25):
  
  from google.colab import drive
  drive.mount('/content/gdrive')
  
  with open("/content/gdrive/My Drive/colab/preprocessedData/XtrainWindowSize" + str(windowSize) + ".npy", 'wb') as outfile:
      np.save(outfile, X_trainPatches)
  with open("/content/gdrive/My Drive/colab/preprocessedData/XtestWindowSize" + str(windowSize) + ".npy", 'wb') as outfile:
      np.save(outfile, X_testPatches)
  with open("/content/gdrive/My Drive/colab/preprocessedData/ytrainWindowSize" + str(windowSize) + ".npy", 'wb') as outfile:
      np.save(outfile, y_trainPatches)
  with open("/content/gdrive/My Drive/colab/preprocessedData/ytestWindowSize" + str(windowSize) + ".npy", 'wb') as outfile:
      np.save(outfile, y_testPatches)
      
      
      
      
# Global Variables
numComponents = 30
windowSize = 5
testRatio = 0.25
saved = True

from google.colab import drive
drive.mount('/content/gdrive')

if saved != True:
  X, y = load_pavia()
  X,pca = applyPCA(X,numComponents=numComponents)
  XPatches, yPatches = createPatches(X, y, windowSize=windowSize)
  X_train, X_test, y_train, y_test = splitTrainTestSet(XPatches, yPatches, testRatio)
  X_train, y_train = oversampleWeakClasses(X_train, y_train)
  X_train = AugmentData(X_train)
  savePreprocessedData(X_train, X_test, y_train, y_test, windowSize = windowSize, 
                      numPCAComponents = numComponents,testRatio = testRatio)
  print(X_train.shape)
  
else:
  X_train = np.load("/content/gdrive/My Drive/colab/preprocessedData/XtrainWindowSize" + str(windowSize) + ".npy")
  y_train = np.load("/content/gdrive/My Drive/colab/preprocessedData/ytrainWindowSize" + str(windowSize) + ".npy")
  X_test = np.load("/content/gdrive/My Drive/colab/preprocessedData/XtestWindowSize" + str(windowSize) + ".npy")
  y_test = np.load("/content/gdrive/My Drive/colab/preprocessedData/ytestWindowSize" + str(windowSize) + ".npy")
  print(X_train.shape)

Mounted at /content/gdrive
(128051, 5, 5, 30)


# Conv. Ladder Net
Architecture: INPUT -> [[CONV -> RELU]*N -> POOL]*M -> [FC -> RELU]*K -> FC

**Params**: N,M,K,filter_size (Array of length N)

**Default**: N=3,M=0,K=1, filter_size=[3*PCA_comp=90, 30, 15]

**For now use only convolution**


In [0]:
import tensorflow as tf
from attributedict.collections import AttributeDict

def train(X,y,X_test=None,y_test=None,N=3,filter_size=[90,30,15],kernel_size=5,
          denoising_cost=[10,1,0.1,0.1,0.1],num_epochs=150,batch_size=200,num_labeled=100,noise_std=0.3,lr=0.02,
          decay_after=15):
  
  tf.set_random_seed(12345)
  
  #Shape of X: (?,WND_SZE,WND_SZE, N_CHANNELS)
  WND_SZE = X.shape[1]
  N_CHANNELS = X.shape[3]
  N_CLASSES = len(np.unique(y))
  #N_CLASSES = 9 #Hardcoded for Pavia Un. and Salinas dataset
  N_EXAMPLES = X.shape[0]
  DEPTH = X.shape[-1]
  
  #Reset the tf default graph
  tf.reset_default_graph()
  
  inputs =  tf.placeholder(tf.float32, shape=(None,WND_SZE,WND_SZE,N_CHANNELS),name='inputs')
  outputs = tf.placeholder(tf.float32, shape=(None,),name='outputs')
  isTrain = tf.placeholder(tf.bool, shape=())
  
  
  #Gamma and beta initialization: Need one gamma (for softmax) and N+K many with different shapes. 
  gamma = tf.Variable(tf.ones([N_CLASSES])) #Take the prev. to last one e.g. 90
  beta = [tf.Variable(tf.zeros([batch_size,kernel_size,kernel_size,filter_size[l]])) for l in range(N)]  
  beta = beta + [tf.Variable(tf.zeros([N_CLASSES]))] #For the last layer
  
  #Prepare a tensorflow dataset
  ds = tf.data.Dataset.from_tensor_slices((X, y))
  ds = ds.shuffle(buffer_size=10, reshuffle_each_iteration=True).batch(batch_size=batch_size, drop_remainder=True).repeat()
  iter = ds.make_one_shot_iterator()
  next = iter.get_next()

  def usetrain():
    inputs = next[0]
    outputs = next[1]
    return inputs, outputs
  def usetest():
    return X_test, y_test

  #Assert X_test is not None. This should be checked when running a session
  input, output = tf.cond(isTrain, usetrain, usetest)
  
  #Helper functions
  join = lambda l, u: tf.concat([l, u], axis=0) #Stack in the depth (batch, height, w, depth)
  labeled = lambda x: x[:num_labeled] if x is not None else x #Use tf.getitem (implicitly)
  unlabeled = lambda x: x[num_labeled:] if x is not None else x
  split_lu = lambda x: (labeled(x), unlabeled(x))
  
  #Running average for the clean pass and the labeled points
  ema = tf.train.ExponentialMovingAverage(decay=0.9999)  # to calculate the moving averages of mean and variance
  #Initialize with shapes (1,kernel_size, kernel_size, filter_size)
  running_mean = [tf.Variable(tf.constant(0.0, shape=[1,kernel_size,kernel_size,f]), trainable=False) for f in filter_size]
  running_mean = running_mean + [tf.Variable(tf.constant(0.0, shape=[N_CLASSES]))]
  running_var = [tf.Variable(tf.constant(1.0, shape=[1,kernel_size,kernel_size,f]), trainable=False) for f in filter_size]
  running_var = running_var + [tf.Variable(tf.constant(1.0, shape=[N_CLASSES]))]
  
  
  def new_activation_dict():
    return AttributeDict({'z': {}, 'h': {}, 's': {}, 'm': {}})
 
  W = tf.Variable(tf.random_normal(shape=[kernel_size**2 * filter_size[-1],N_CLASSES]))
  V = tf.Variable(tf.random_normal(shape=[N_CLASSES, kernel_size**2 * filter_size[-1]]))
  
  L = N+2 #Convs+In+Soft
  
  
  def g(z_lat, u):
    shape = tf.shape(u)[1:] #Don't take the batch size as a dimension 
    wi = lambda inits, name: tf.Variable(inits * tf.ones(shape), name=name)
    a1 = wi(0., 'a1')
    a2 = wi(1., 'a2')
    a3 = wi(0., 'a3')
    a4 = wi(0., 'a4')
    a5 = wi(0., 'a5')

    a6 = wi(0., 'a6')
    a7 = wi(1., 'a7')
    a8 = wi(0., 'a8')
    a9 = wi(0., 'a9')
    a10 = wi(0., 'a10')

    mu = a1 * tf.sigmoid(a2 * u + a3) + a4 * u + a5
    v = a6 * tf.sigmoid(a7 * u + a8) + a9 * u + a10

    z_est = (z_lat - mu) * v + mu
    return z_est
  
  #Encoder
  def encoder(input, noise_std):
    #Apply noise to the input
    h = tf.cast(input,tf.float32) + tf.random_normal(tf.shape(input)) * noise_std #Normal noise 0 mean 1 std
    d = AttributeDict() #This is what we will return. It will contain all the information we need
    d.unlabeled = new_activation_dict()
    d.labeled = new_activation_dict()
    d.unlabeled.z[0] = unlabeled(h)
    d.labeled.z[0] = labeled(h)
    
    for i in range(1,L): #Go through the convolutional layers, if we are at i==N+1, we need to flatten and apply W
      d.labeled.h[i-1], d.unlabeled.h[i-1] = split_lu(h)
      
      if i==L-1:
        z = tf.layers.flatten(h)
        z = tf.matmul(z,W)
      else:
        #Compute new z by applying convolution followed by ReLU after normalization
        z = tf.layers.conv2d(h,filters=filter_size[i-1], kernel_size=kernel_size, padding='same')
      #Shape: (?,5,5,filter_size)
      #Normalize
      z_lbld, z_unlbld = split_lu(z)
      
      keep_dims = True
      if i==L-1:
        keep_dims = False
        
      m_unlbld, s_unlbld = tf.nn.moments(z_unlbld, axes=[0], keep_dims=keep_dims) #Compute along depth
      m_lbld, s_lbld = tf.nn.moments(z_lbld, axes=[0], keep_dims=keep_dims)
      #Shape: (1,5,5,filter_size)
      
      if noise_std == 0: #Clean pass
        #Update the running averages and get the mean and variance of the labeled points again
        assign_mean = running_mean[i-1].assign(m_lbld)
        assign_var = running_var[i-1].assign(s_lbld)
        with tf.control_dependencies([assign_mean, assign_var]):
          ema.apply([running_mean[i-1], running_var[i-1]])
          m_lbld = ema.average(running_mean[i-1])
          s_lbld = ema.average(running_var[i-1])

          
      z = join(
        (z_lbld-m_lbld) / tf.sqrt(s_lbld + 1e-10),
        (z_unlbld-m_unlbld) / tf.sqrt(s_unlbld + 1e-10))
      
      if noise_std > 0:
        z += tf.random_normal(tf.shape(z)) * noise_std
        
      z_lateral = z
      
      if i==L-1: #We need to apply softmax and multiply with gamma
        z += beta[i-1]
        z = tf.multiply(z, gamma)
        h = tf.nn.softmax(z)
      else:  
        #Now apply activation. But before we apply the activation, add beta and multiply
        #with gamma. Gamma is not used for ReLU. We apply Gamma for the softmax layer.
        z += beta[i-1] #i starts at 1, but beta starts at 0
        #Apply ReLU
        h = tf.nn.relu(z) #h gets assigned at the beginning of the for loop
      
      #Now save the variables: z_lateral, m_unlbld, s_unlbld, h
      d.labeled.z[i], d.unlabeled.z[i] = split_lu(z_lateral) #The real z has been compromised
      d.unlabeled.s[i] = s_unlbld
      d.unlabeled.m[i] = m_unlbld
      
    #Get the last h.
    d.labeled.h[i], d.unlabeled.h[i] = split_lu(h)
    
    return h, d
  #End encoder
  
  #Get the clean run
  y_clean, clean = encoder(input, noise_std=0.0)
  #Get the corrupted encoder run
  y_corrupted, corr = encoder(input, noise_std=noise_std)
  
  #Use this to store the z_est etc.
  est = new_activation_dict()
  
  #Decoder path
  filter_dims = [DEPTH] + filter_size
  #Start at index N+1 and go through index 0, N=3
  cost_recon = []
  for i in np.arange(L)[::-1]: #Start from L+1 --> 0, L+1 = N+2 = 6, 30-90-30-15-9
    #Get all the information we need
    z_corr = corr.unlabeled.z[i]
    z_clean = clean.unlabeled.z[i]
    
    #We don't have running mean and variance for the alreadz standardized input. TODO: Correct?
    if i != 0:
      z_clean_s = clean.unlabeled.s[i]
      z_clean_m = clean.unlabeled.m[i]
      
    if i==L-1: #The top level
      #Just normalize the (100,9) output
      ver = corr.unlabeled.h[i]
    elif i==L-2: #Apply the matrix V
      ver = tf.matmul(est.z.get(i+1), V) #This produces a (100,375)
      #Now reshape to [batch_size, kernel, kernel, filter[-1]]
      ver = tf.reshape(ver, shape=corr.unlabeled.z[N].shape)
    else:
      #Deconvolve. This is just a convolution to a new filter size. We leave the kernel size untouched.
      ver = tf.layers.conv2d(est.z.get(i+1),filters=filter_dims[i], kernel_size=kernel_size, padding='same')
  
    #Now normalize them
    keep_dims = True
    if i==L-1: #Only for the first dimension don't keep the dimension since it is (?,), not (?,5,5,?)
      keep_dims = False
        
    m, s = tf.nn.moments(ver, axes=[0], keep_dims=keep_dims) #Compute along depth
    ver = (ver-m) / tf.sqrt(s + 1e-10)
    
    #Now apply g to get z_est, g(z_corr_from_encoder, ver (u in the paper))
    z_est = g(z_corr, ver)
    
    #Now normalize using the clean mean and clean variance, but only if i != 0
    if i != 0:
      z_est_norm = (z_est - z_clean_m) / tf.sqrt(z_clean_s + 1e-10)
    else: z_est_norm = z_est
    
    #Now compute the cost and append the weighted cost
    c_tmp = tf.nn.l2_loss(z_clean-z_est_norm) * denoising_cost[i]
    cost_recon.append(c_tmp)
    est.z[i] = z_est_norm
    
  target = labeled(tf.one_hot(tf.cast(output,tf.int32),depth=N_CLASSES))
  
  y_corrupted = labeled(y_corrupted)
  supervised_cost = -tf.reduce_mean(tf.reduce_sum(target*tf.log(y_corrupted), 1))
  unsupervised_cost = tf.add_n(cost_recon)
  loss = supervised_cost + unsupervised_cost
  
  
  
  prediction_cost = -tf.reduce_mean(tf.reduce_sum(target*tf.log(labeled(y_clean)), 1),name='pred_cost')
  correct_prediction = tf.equal(tf.argmax(labeled(y_clean), 1), tf.argmax(target, 1), name='correct_prediction')
  accuracy = tf.multiply(tf.reduce_mean(tf.cast(correct_prediction, "float")),tf.constant(100.0),name='accuracy')
  
  learning_rate = tf.Variable(lr, trainable=False)
  train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)
  
  
  debug = accuracy

      
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    n_iter = int(N_EXAMPLES/batch_size)
    for epoch in range(num_epochs):
      for i in range(n_iter):
        _, acc = sess.run([train_step, accuracy], feed_dict={isTrain: True})
        #if i % 100 == 0: print("Training accuracy: %s" % acc)
          
      acc = sess.run(accuracy, feed_dict={isTrain: True})
      print("Epoch No.: %s ACC: %s" % (epoch, acc))
        
        
        
        
        

In [6]:
train(X_train,y_train,X_test,y_test,num_epochs=10)

Epoch No.: 0 ACC: 11.0
Epoch No.: 1 ACC: 17.0
Epoch No.: 2 ACC: 10.0


KeyboardInterrupt: ignored